In [105]:
import pandas as pd
import sqlite3

In [106]:
name_bd = '../checking-logs.sqlite.sqlite_copy'
connect = sqlite3.connect(name_bd)

In [107]:
query = """
SELECT ch.uid, ch.labname, ch.timestamp AS first_commit_ts, pageviews.datetime AS first_view_ts
FROM checker ch LEFT JOIN pageviews ON ch.uid = pageviews.uid
WHERE
ch.status = 'ready' AND
ch.numTrials = 1 AND
ch.labname in ("laba04", "laba04s", "laba05", "laba06", "laba06s", "project1") AND
ch.uid LIKE 'user_%' AND ch.uid != 'admins' AND
(pageviews.datetime = (SELECT MIN(pageviews.datetime)
                           FROM pageviews
                           WHERE uid=ch.uid)
     OR pageviews.datetime IS NULL)
"""
datamart = pd.io.sql.read_sql(query, connect, parse_dates=['first_commit_ts', 'first_view_ts'])
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


In [108]:
test = datamart[datamart['first_view_ts'].notna()]
test.head()

,uid,labname,first_commit_ts,first_view_ts
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447


In [109]:
control = datamart[datamart['first_view_ts'].isna()].reset_index(drop=True)
control['first_view_ts'].fillna(test['first_view_ts'].mean(), inplace=True)
control.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783552
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783552
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783552
3,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783552
4,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783552


In [110]:
test.to_sql('test', connect)
control.to_sql('control', connect)

ValueError: Table 'test' already exists.

In [112]:
connect.close()